<h1 style="text-align:center; LINE-HEIGHT:35px">Advanced SQL Queries on the Australian Open <br>Tennis Tournament Database</h1>
<hr>
## Introduction
In this notebook, we will write some advanced SQL queries to obtain some data from the Australian Open Tennis Tournament Database*. We will first provide the background required to understand how the database is structured and then start with the queries.
## Background Information
### TOURNAMENT RULES
#### GAME
A game consists of a sequence of points played with the same player serving. The
running score of each game is described in a manner peculiar to tennis: scores from zero
to three points are described as "0", "15", "30", and "40", respectively. A game is won by
the first player to have won at least four points in total and at least two points more than
the opponent. If at least three points have been scored by each player, making the
player's scores equal at forty apiece, the score is not called out as "forty-forty", but
rather as "deuce". If at least three points have been scored by each side and a player has
one more point than his opponent, the score of the game is "advantage" for the player in
the lead. If the same player also wins the next point, that player wins the “Game”; if the
opposing player wins the next point, the score is again “Deuce”. A player needs to win
two consecutive points immediately after “Deuce” to win the “Game”.
#### SET
A set consists of a sequence of games played with service alternating between games,
ending when the count of games won meets certain criteria. Typically, a player wins a
set by winning at least six games and at least two games more than the opponent. If one
player has won six games and the opponent five, an additional game is played. If the
leading player wins that game, the player wins the set 7–5. If the trailing player wins the
game, the set score becomes 6-6 and a tie-break is played. A tie-break allows one player
to win one more game and thus the set, to give a final set score of 7–6.
#### MATCH 
A match consists of a sequence of 3 sets for women. The outcome is determined through a best of three sets system. In other words, if a player wins 2 sets, she wins the match.
### Relational Database
The [database file](https://github.com/Mo-Saif/SQL-Notebook/blob/master/AusOpen.db?raw=true) used in this notebook is made with SQLite and it has the following design:

<img src="https://raw.githubusercontent.com/Mo-Saif/SQL-Notebook/master/tables_and_relationships.png" alt="Tables" width="800">
### Queries

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('AusOpen.db')
c = conn.cursor()

#### Query 1
Write a query that shows the player having the highest number of sets won through tiebreaks, i.e. Tiebreak Queen of the tournament. The query should show the license no, name, surname of the Tiebreak Queen and the number of tiebreaks she won in the tournament.

In [2]:
# Q1
c.execute('''
SELECT sets.winner_licence_no, player.name, player.surname, COUNT(sets.tiebreak) AS CountOfTiebreak
FROM player INNER JOIN sets ON player.player_licence_no = sets.winner_licence_no
GROUP BY sets.winner_licence_no, player.name, player.surname, sets.tiebreak
HAVING ((sets.tiebreak)='YES')
ORDER BY Count(sets.tiebreak) DESC
LIMIT 1;
''')
c.fetchall()

[(1120308, 'Angelique', 'Kerber', 3)]

#### Query 2
Write a query that shows the number of players attending the tournament in 2017 from a given country. The country code will be entered by the user.

In [3]:
# Q2
cc = ['DE']
c.execute('''
SELECT country.country_name, COUNT(player.player_licence_no) AS [countofplayer licence no]
FROM country INNER JOIN player ON country.country_code = player.nationality
GROUP BY country.country_name, country.country_code
HAVING country.country_code= ?;
''',cc)
c.fetchall()

[('Germany', 11)]

#### Query 3
Most matches in the tournament are played between two different country’s players.
We are interested in obtaining a statistic that shows for a given
country pair, the percentage of the matches won by the players from a specific country
available on the tournament’s website.
The codes of the countries in the country pair and the country code that
the percentage of winning is desired to be seen will be entered by the user. (i.e. user will
enter the country codes of the country pair respectively, then enter the country code that
he/she wants to see the percentage of winning)

In [4]:
# Q3
c1='DE' # country 1 code
c2='US' # country 2 code
sc='DE' # selected country
c.execute('''
SELECT ? AS [country 1], ? AS [country 2], ? AS [selected country], (sub3.count*1.0/(sub1.count+sub2.count)) AS percentage
FROM (SELECT COUNT(*) AS count 
      FROM (SELECT winner.nationality, loser.nationality 
            FROM match, player AS winner, player AS loser 
            WHERE match.winner_licence_no = winner.player_licence_no AND match.loser_licence_no = loser.player_licence_no 
                AND winner.nationality = ? AND loser.nationality = ?)  ) AS sub1,
     (SELECT COUNT(*) AS count 
      FROM (SELECT winner.nationality, loser.nationality 
            FROM match, player AS winner, player AS loser 
            WHERE match.winner_licence_no = winner.player_licence_no AND match.loser_licence_no = loser.player_licence_no 
                AND winner.nationality = ? AND loser.nationality = ?) ) AS sub2,
     (SELECT COUNT(*) AS count 
      FROM (SELECT winner.nationality, loser.nationality 
            FROM match, player AS winner, player AS loser 
            WHERE match.winner_licence_no = winner.player_licence_no 
                AND match.loser_licence_no = loser.player_licence_no AND winner.nationality = ? 
                AND loser.nationality = (SELECT country_code FROM country 
                                         WHERE country_code  <> ? AND country_code IN (?,?))) ) AS sub3;
''',(c1, c2, sc, c1, c2, c2, c1, sc, sc, c1, c2))
c.fetchall()

[('DE', 'US', 'DE', 0.6666666666666666)]

#### Query 4
Let's investigate whether there is a relation
between the age of the players who played in the tournament and corresponding win
rates. Create a query that shows the win rate of the players over a given age.
Age will be entered by the user.

In [5]:
# Q4
age=22
c.execute('''
SELECT (t1.counter*1.0/(t2.counter+t3.counter)) AS winrate
FROM (SELECT COUNT(*) AS counter 
      FROM match, player AS winner 
      WHERE match.winner_licence_no = winner.player_licence_no 
          AND winner.year_of_birth <= 2017- ?)  AS t1, 
     (SELECT COUNT(*) AS counter 
      FROM match, player AS winner, player AS loser 
      WHERE match.winner_licence_no = winner.player_licence_no 
          AND match.loser_licence_no = loser.player_licence_no 
          AND (winner.year_of_birth <= 2017-? OR loser.year_of_birth <= 2017 - ?))  AS t2,
     (SELECT COUNT(*) AS counter 
      FROM match, player AS winner, player AS loser 
      WHERE match.winner_licence_no = winner.player_licence_no 
          AND match.loser_licence_no = loser.player_licence_no 
          AND (winner.year_of_birth <= 2017-? AND loser.year_of_birth <= 2017 - ?))  AS t3;
''', (age, age, age, age, age))
c.fetchall()

[(0.5043103448275862,)]

#### Query 5
Create a query that returns the name-surname of the player in a single field and the
percentage of advantages turned into a game.
The user should be able to search for the part of the name or surname of the player.

In [6]:
# Q5
name = ['denisa']
c.execute('''
SELECT (name || ' ' || surname) AS fullname, (t1.count*1.0/(t2.loseradv + t3.winneradv)) AS percentage
FROM player, 
    (SELECT COUNT(*) AS count, player.player_licence_no AS pln 
     FROM game, player 
     WHERE game.winner_licence_no = player.player_licence_no 
         AND advantages_of_winner > 0 GROUP BY player_licence_no) AS t1, 
    (SELECT player_licence_no AS pln2, SUM(advantages_of_loser) AS loseradv
     FROM game, match, player
     WHERE game.match_id = match.match_id AND player_licence_no <> game.winner_licence_no 
         AND (player_licence_no = match.winner_licence_no OR player_licence_no = match.loser_licence_no)
     GROUP BY player_licence_no)  AS t2, 
    (SELECT game.winner_licence_no AS pln3, SUM(game.advantages_of_winner) AS winneradv
     FROM game
     GROUP BY game.winner_licence_no)  AS t3
     WHERE (((name || ' ' || surname) LIKE "%" || ? || "%")) AND player.player_licence_no = t1.pln 
         AND player.player_licence_no = t2.pln2 AND player.player_licence_no = t3.pln3;
''', (name))
c.fetchall()

[('Denisa Allertova', 0.17204301075268819)]

In [7]:
c.close()
conn.close()


\*Data Source: Middle East Technical University, Industrial Engineering Department, IE304 Management Information Systems Term Project